In [5]:
from flask import Flask, render_template, Response, request, redirect, url_for
import cv2
import os
from ultralytics import YOLO
import math

In [6]:
def image_detect(image_path):
    model = YOLO("runs/detect/train/weights/best.pt")
    classNames = ["bouledogue","blacktip","whitetip"]

    img = cv2.imread(image_path)
    if img is None:
        return None
    results = model(img)
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = math.ceil((box.conf[0] * 100)) / 100
            if conf > 0.5:
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cls = int(box.cls[0])
                class_name = classNames[cls]
                label = f'{class_name} : {conf}'
                t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                c2 = x1 + t_size[0], y1 - t_size[1] - 3
                cv2.rectangle(img, (x1,y1), c2, [0,255,0], -1, cv2.LINE_AA)
                cv2.putText(img, label, (x1, y1-2), 0, 1, [255,255,255], thickness=1, lineType=cv2.LINE_AA)
    result_path = image_path.replace('uploads', 'results')
    cv2.imwrite(result_path, img)
    return result_path

In [7]:
def video_detection(path):
    """
    Detects objects in a video using YOLO model.

    Args:
        path (str): The path to the video file.

    Yields:
        numpy.ndarray: The processed frame with object detection annotations.
    """
    cap = cv2.VideoCapture(path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    
    model = YOLO("runs/detect/train/weights/best.pt")
    classNames = ["bouledogue", "blacktip", "whitetip"]
    
    while True:
        success, img = cap.read()
        results = model(img, stream=True)
        
        for r in results:
            boxes = r.boxes
            for box in boxes:
                conf = math.ceil((box.conf[0] * 100)) / 100
                if conf > 0.5:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cls = int(box.cls[0])
                    class_name = classNames[cls]
                    label = f'{class_name} : {conf}'
                    t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                    c2 = x1 + t_size[0], y1 - t_size[1] - 3
                    cv2.rectangle(img, (x1, y1), c2, [0, 255, 0], -1, cv2.LINE_AA)
                    cv2.putText(img, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
        
        yield img

In [8]:
app = Flask(__name__) 

app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['RESULT_FOLDER'] = 'static/results/'

def generate_frames(path = ''):
    yolo_output = video_detection(path)
    for detection in yolo_output :
        ref, buffer = cv2.imencode('.jpg', detection)
        frame = buffer.tobytes()
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/gallery')
def gallery():
    # Liste les fichiers dans le dossier static/results
    images = os.listdir('static/results')
    # Filtre les fichiers pour n'inclure que les images
    images = [image for image in images if image.endswith(('jpg', 'jpeg', 'png', 'gif', 'JPG'))]
    return render_template('gallery.html', images=images)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_image():
    file = request.files['file']
    if file :
        filename = file.filename
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        image_detect(filepath)
        return render_template('index.html', filename=filename)
    else:
        return redirect(request.url)

@app.route('/video')
def video():
    #return Response(generate_frames(path=mavideo), mimetype='multipart/x-mixed-replace; boundary=frame')
    pass
    
@app.route('/webcam')
def webcam():
    return Response(generate_frames(path=0), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/display/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename=f'results/{filename}', code = 301))

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/May/2024 22:21:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:14] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /gallery HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/IMG_4408.jpg HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4155206.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4155211.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4155212.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4155271.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4165376.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4165345.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /static/results/P4155283.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:21:15] "GET /s


0: 480x640 1 bouledogue, 149.0ms
Speed: 2.5ms preprocess, 149.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


127.0.0.1 - - [15/May/2024 22:21:18] "GET /webcam HTTP/1.1" 200 -



0: 480x640 1 bouledogue, 115.6ms
Speed: 2.0ms preprocess, 115.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bouledogue, 110.0ms
Speed: 1.0ms preprocess, 110.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 bouledogues, 111.2ms
Speed: 1.0ms preprocess, 111.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bouledogue, 108.8ms
Speed: 2.5ms preprocess, 108.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 135.8ms
Speed: 1.5ms preprocess, 135.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bouledogue, 160.2ms
Speed: 2.0ms preprocess, 160.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 169.6ms
Speed: 1.1ms preprocess, 169.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bouledogue, 162.0ms
Speed: 2.2ms preprocess, 162.0ms in

...

0: 480x640 (no detections), 163.6ms
Speed: 2.1ms preprocess, 163.6ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)


127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/IMG_4408.jpg HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155206.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155211.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155212.JPG HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155271.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155277.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4165345.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155283.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4165376.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4165406.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /static/results/P4155278.JPG HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 22:22:38] "GET /


0: 480x640 (no detections), 129.8ms
Speed: 3.0ms preprocess, 129.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


127.0.0.1 - - [15/May/2024 22:23:05] "GET /webcam HTTP/1.1" 200 -



0: 480x640 (no detections), 131.0ms
Speed: 0.0ms preprocess, 131.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 126.9ms
Speed: 2.6ms preprocess, 126.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 118.1ms
Speed: 0.0ms preprocess, 118.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 whitetip, 114.8ms
Speed: 0.0ms preprocess, 114.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 whitetip, 122.4ms
Speed: 0.0ms preprocess, 122.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 whitetip, 121.2ms
Speed: 1.5ms preprocess, 121.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 114.0ms
Speed: 0.0ms preprocess, 114.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 118.2ms
Speed: 0.0ms preprocess, 118.2ms 